In [ ]:
!pip install langchain-fireworks

In [85]:
import os
import sys
import logging
import time

from dotenv import load_dotenv
from langchain_sambanova import ChatSambaStudio, ChatSambaNovaCloud
from langchain_fireworks import ChatFireworks 
from langchain_core.prompts import PromptTemplate

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

load_dotenv(os.path.join(repo_dir, '.env'), override=True)

True

In [86]:
# chat models to evaluate

llm_sambanova_cloud = ChatSambaNovaCloud(
    max_tokens=1200, 
    model='Meta-Llama-3.1-8B-Instruct'
    )

llm_sambastudio = ChatSambaStudio(
    max_tokens=1200, 
    model='Meta-Llama-3.1-8B-Instruct'
    )

llm_provider1 = ChatFireworks(
    max_tokens=1200, 
    model="accounts/fireworks/models/llama-v3p1-8b-instruct"
)

In [87]:
# guardrail models to evaluate

guardrails_sambanova_cloud = ChatSambaNovaCloud(
    max_tokens=1200, 
    model='Meta-Llama-Guard-3-8B'
    )

guardrails_sambastudio = ChatSambaStudio(
    max_tokens=1200, 
    model='Meta-Llama-Guard-3-8B'
    )

guardrails_provider1 = ChatFireworks(
    max_tokens=1200, 
    model='accounts/fireworks/models/llama-guard-3-8b'
)

In [88]:
unsafe_categories = """S1: Violent Crimes.
AI models should not create content that enables, encourages, or excuses the commission of violent crimes. Examples of violent crimes include, but are not limited to:
- Unlawful violence toward people (ex: terrorism, genocide, murder, hate crimes, child abuse, assault, battery, kidnapping)
- Unlawful violence toward animals (ex: animal abuse)
S2: Non-Violent Crimes.
AI models should not create content that enables, encourages, or excuses the commission of non-violent crimes. Examples of non-violent crimes include, but are not limited to:
- Financial crimes (ex: fraud, scams, money laundering)
- Property crimes (ex: burglary, robbery, arson, vandalism)
- Drug crimes (ex: creating or trafficking narcotics)
- Weapons crimes (ex: producing unlicensed firearms)
- Cyber crimes (ex: hacking, spyware, malware)
S3: Sex Crimes.
AI models should not create content that enables, encourages, or excuses the commission of sex crimes. Examples of sex crimes include, but are not limited to:
- Human trafficking
- Sexual assault (ex: rape)
- Sexual harassment (ex: groping)
- Lewd conduct
S4: Child Exploitation.
AI models should not create content that depicts child nudity or that enables, encourages, excuses, or depicts the sexual abuse of children.
S5: Specialized Advice.
AI models should not create content that contains specialized financial, medical, or legal advice.
S6: Privacy.
AI models should not create content that contains sensitive, personal information about private individuals.
S7: Intellectual Property.
AI models should not create content that violates the intellectual property rights of third parties.
S8: Indiscriminate Weapons.
AI models should not create content that enables the creation of weapons of mass destruction. Examples of such weapons include, but are not limited to:
- Chemical weapons (ex: nerve gas)
- Biological weapons (ex: anthrax)
- Radiological weapons (ex: salted bombs)
- Nuclear weapons (ex: atomic warheads)
- High-yield explosive weapons (ex: cluster munitions)
S9: Hate.
AI models should not create content that is hateful toward people on the basis of their protected characteristics (race, color, ethnicity, national origin, disability, religious affiliation, caste, sexual orientation, sex, gender identity and serious disease). AI models should not create content that perpetuates negative stereotypes.
S10: Self-Harm.
AI models should not create content that enables, encourages, or excuses acts of intentional self-harm. Examples of such actions include, but are not limited to:
- Suicide
- Self-injury (ex: cutting)
- Disordered eating
S11: Sexual Content.
AI models should not create depictions of nude adults or content that contains erotic descriptions or explicit depictions of sex acts.
<END UNSAFE CONTENT CATEGORIES>
)"""

def evaluate_guardrails(guardrails_model, input, role, unsafe_categories=unsafe_categories):
    guardrails_prompt_template = PromptTemplate.from_template(
        """<|begin_of_text|>[INST] Task: Check if there is unsafe content in '{role}' messages in conversations according our safety policy with the below categories.

        <BEGIN UNSAFE CONTENT CATEGORIES>
        {unsafe_categories}
        <END UNSAFE CONTENT CATEGORIES>

        <BEGIN CONVERSATION>

        {conversation}
        <END CONVERSATION>

        Provide your safety assessment for ONLY THE LAST {role} in the above conversation:
        - First line must read 'safe' or 'unsafe'.
        - If unsafe, a second line must include a comma-separated list of violated categories. [/INST]"""
    )
    if isinstance(input, str):
        conversation = f"{role.capitalize()}: {input}\n\n"
    elif isinstance(input, list):
        conversation = ""
        for turn in conversation:
            conversation += f"{turn['role'].capitalize()}: {turn['content']}\n\n"
        
    prompt = guardrails_prompt_template.format(
        unsafe_categories = unsafe_categories,
        conversation= conversation,
        role = role
    )
    
    return guardrails_model.invoke(prompt)

In [113]:
def invoke_with_guardrails_metrics(llm, guardrails_model, prompt):
    client_latencies={}
    client_latencies["start_time"] = time.time()
    guardrails_user_response = evaluate_guardrails(guardrails_model, prompt, role = "user")
    client_latencies["user_guardrails_end_time"] = time.time()
    client_latencies["user_guardrails_time"] = client_latencies["user_guardrails_end_time"]-client_latencies["start_time"]
    if guardrails_user_response.content.strip().split("\n")[0] == "unsafe":
        logging.warning("User input {} has violated guardrails: {}".format(prompt, guardrails_user_response.content.replace('\n', ' ')))
        return guardrails_user_response, None, None, client_latencies
    llm_response = llm.invoke(prompt)
    client_latencies["llm_generation_end_time"] = time.time()
    client_latencies["llm_generation_time"] = client_latencies["llm_generation_end_time"]-client_latencies["user_guardrails_end_time"]
    guardrails_assistant_response = evaluate_guardrails(
        guardrails_model,
        [{
            "user": prompt, 
            "assistant": llm_response.content 
        }],
        role = "assistant"
        )
    client_latencies["assistant_guardrails_end_time"] = time.time()
    client_latencies["assistant_guardrails_time"] = client_latencies["assistant_guardrails_end_time"]-client_latencies["llm_generation_end_time"]
    client_latencies["end_time"] = client_latencies["assistant_guardrails_end_time"]
    client_latencies["e2e_time"] = client_latencies["end_time"]-client_latencies["start_time"]
    if guardrails_assistant_response.content.strip().split("\n")[0] == "unsafe":
        logging.warning("Model response {} has violated guardrails: {}".format(llm_response.content, guardrails_assistant_response.content.replace('\n', ' ')))
    return guardrails_user_response, llm_response, guardrails_assistant_response, client_latencies

In [114]:
invoke_with_guardrails_metrics(llm_sambanova_cloud, guardrails_sambanova_cloud, "hi")

(AIMessage(content='\n\nsafe', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'usage': {'acceptance_rate': 1, 'completion_tokens': 2, 'completion_tokens_after_first_per_sec': 43.19527090349224, 'completion_tokens_after_first_per_sec_first_ten': 0, 'completion_tokens_per_sec': 39.34065563007081, 'end_time': 1738690555.9188704, 'is_last_response': True, 'prompt_tokens': 911, 'start_time': 1738690555.8680325, 'stop_reason': 'stop', 'time_to_first_token': 0.02768731117248535, 'total_latency': 0.05083799362182617, 'total_tokens': 913, 'total_tokens_per_sec': 17959.00929512733}, 'model_name': 'Meta-Llama-Guard-3-8B', 'system_fingerprint': 'fastcoe', 'created': 1738690555}, id='c064f420-42c1-4674-aaf0-ca7e067ab77f'),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'usage': {'acceptance_rate': 1, 'completion_tokens': 7, 'completion_tokens_after_first_per_sec': 280.8779758250834, 'completion_tokens_after_firs

In [115]:
invoke_with_guardrails_metrics(llm_sambastudio, guardrails_sambastudio, "hi")

(AIMessage(content='\n\nsafe', additional_kwargs={}, response_metadata={'id': 'item0', 'partial': False, 'value': {'completion': '\n\nsafe', 'completion_tokens_count': 2, 'logprobs': {'text_offset': [], 'top_logprobs': []}, 'model_execution_time': 0.04898953437805176, 'prompt': " <|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nTask: Check if there is unsafe content in 'User' messages in conversations according our safety policy with the below categories.\n\n<BEGIN UNSAFE CONTENT CATEGORIES>\nS1: Violent Crimes.\nS2: Non-Violent Crimes.\nS3: Sex Crimes.\nS4: Child Exploitation.\nS5: Defamation.\nS6: Specialized Advice.\nS7: Privacy.\nS8: Intellectual Property.\nS9: Indiscriminate Weapons.\nS10: Hate.\nS11: Self-Harm.\nS12: Sexual Content.\nS13: Elections.\nS14: Code Interpreter Abuse.\n<END UNSAFE CONTENT CATEGORIES>\n\n<BEGIN CONVERSATION>\n\nUser: <|begin_of_text|>[INST] Task: Check if there is unsafe content in 'user' messages in conversations according our safety policy

In [116]:
invoke_with_guardrails_metrics(llm_provider1, guardrails_provider1, "hi")

(AIMessage(content='\n\nsafe', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 911, 'total_tokens': 914, 'completion_tokens': 3}, 'model_name': 'accounts/fireworks/models/llama-guard-3-8b', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e77c617-1519-477d-a6ec-697d32e61961-0', usage_metadata={'input_tokens': 911, 'output_tokens': 3, 'total_tokens': 914}),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 12, 'total_tokens': 20, 'completion_tokens': 8}, 'model_name': 'accounts/fireworks/models/llama-v3p1-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba753a53-14b0-40a4-9053-90d823adbeab-0', usage_metadata={'input_tokens': 12, 'output_tokens': 8, 'total_tokens': 20}),
 AIMessage(content='\n\nsafe', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 907, 'total_tokens': 910, 'completion_tokens